# # 웹스크랩

## - 소스 가져오기

In [1]:
import requests

In [2]:
hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'}
res = requests.get('https://www.coupang.com/np/rocketdelivery?page=1', headers = hdr)

## - 파싱하기

In [3]:
from bs4 import BeautifulSoup

In [4]:
# html.parser 및 lxml 2가지 사용 가능
soup = BeautifulSoup(res.text, 'html.parser')

In [8]:
# find/find_all : tag기준 검색
#       ('tag명', id='id명', class_='class명')
# select_one/select
#       ('tag명' | '#id명' | '.class명')
#       ('tag명 > tag명.class명' | '#id명 > tag명.class명' | 'tag명[attr명=값]')

# crom browser의 기능 활용
ul_tag = soup.select_one('#productList')

In [10]:
#li_list = ul_tag.find_all('li', class_='baby-product')
li_list = ul_tag.select('li.baby-product')
len(li_list)

60

In [11]:
result_list = []

for li_tag in li_list:
    # result_list.append([li_tag.select_one('a > dl > dd > div.name').text.strip(),
    #                     li_tag.select_one('a > dl > dd > div.price-area > div.price-wrap > div.price > em > strong').text.strip().replace(',', ''),
    #                     li_tag.select_one('a > dl > dd > div.other-info > div.rating-star > span.rating-total-count').text.strip()[1:-1] ])

    # full_path를 사용하지 않아도 됨
    result_list.append([li_tag.select_one('div.name').text.strip(),
                        li_tag.select_one('div.price > em > strong').text.strip().replace(',', ''),
                        li_tag.select_one('span.rating-total-count').text.strip()[1:-1] ])

result_list[:5]

[['곰곰 2019년 소중한 우리 쌀, 10kg, 1개', '25900', '41855'],
 ['곰곰 단백질바, 50g, 12개입', '11490', '1352'],
 ['곰곰 고소한 검은콩 칼슘 두유, 190ml, 24개', '7990', '3062'],
 ['코멧 저자극 시그니처 아기 물티슈 엠보싱 캡형, 100매, 10팩', '10690', '9400'],
 ['포테이토 크리스프 스낵 사워크림맛 50개입, 1kg, 2개', '11320', '266']]

In [12]:
import pandas as pd

In [13]:
df = pd.DataFrame(result_list, columns=['상품명', '가격', '추천도'])
df.head()

,상품명,가격,추천도
0,"곰곰 2019년 소중한 우리 쌀, 10kg, 1개",25900,41855
1,"곰곰 단백질바, 50g, 12개입",11490,1352
2,"곰곰 고소한 검은콩 칼슘 두유, 190ml, 24개",7990,3062
3,"코멧 저자극 시그니처 아기 물티슈 엠보싱 캡형, 100매, 10팩",10690,9400
4,"포테이토 크리스프 스낵 사워크림맛 50개입, 1kg, 2개",11320,266


In [10]:
df.to_csv('coupang_rocket.csv', index=False)

# # Open API 사용

## - 헤더정보 설정

In [14]:
header = {
    'X-Naver-Client-Id': 'Kt9JEg3vtZzC36UOTcyX',
    'X-Naver-Client-Secret': 'HrAlWytIVi'
}

## - 파라메터 : 검색어

In [24]:
data = {
    'query': '부산 여행',
    'display': 100
}

## - 호출

In [25]:
import requests

In [26]:
res = requests.get('https://openapi.naver.com/v1/search/blog.json',
                   headers=header, params=data)

## - 결과 데이터 파싱

In [27]:
import pandas as pd

In [28]:
# http://jsonviewer.stack.hu/
# Json 구문을 정리해 주는 사이트
result = res.json()
result

{'lastBuildDate': 'Thu, 12 Mar 2020 22:32:42 +0900',
 'total': 2409427,
 'start': 1,
 'display': 100,
 'items': [{'title': '<b>부산여행</b> 필수코스 / 문화양곱창 / 부산맛집 추천',
   'link': 'https://blog.naver.com/kpwee?Redirect=Log&logNo=221768253567',
   'description': '<b>부산여행</b> 가시는 분들은 문화양곱창 꼭 들리셔서 여행의 특별한 추억 남기시길 추천드려요. 맛과 로컬분위기 + 부산 이모님과의 특별한 추억 등 이곳에서만 경험할 수 있는 색다름이 있으니까요. 친구에게 받은... ',
   'bloggername': '푸파의 청춘일기',
   'bloggerlink': 'https://blog.naver.com/kpwee',
   'postdate': '20200111'},
  {'title': '알차게 다녀온 <b>부산여행</b>',
   'link': 'https://blog.naver.com/armful31?Redirect=Log&logNo=221738427844',
   'description': '오래간만에 다녀온 <b>부산여행</b>은 부산 숨은 곳을 다녀보는 컨셉! 예전에는 늘 할머니를 뵈러 가는게 제일... 이번 여행에서 젤 고생 많이 하신건 삼촌ㅠㅠㅠㅠ 엄마나 저나 운전을 못하다 보니까 대중교통을 타고... ',
   'bloggername': '미쓰 룰루랄라:)',
   'bloggerlink': 'https://blog.naver.com/armful31',
   'postdate': '20191216'},
  {'title': ':: <b>부산여행</b> :: 부산 동백섬 겨울 여행으로 추천드립니다',
   'link': 'https://blog.naver.com/marzanara?Redirect=Log&logNo=221763836140',
   'de

In [29]:
# 방법1. 리스트 만들어서 무식하게 처리
to_csv_list = []

for item in result['items']:
    to_csv_list.append([
        item['postdate'], item['title'],
        item['description'], item['bloggername']
    ])


df = pd.DataFrame(to_csv_list, columns=['날짜', '제목', '설명', '블로거'])
df.head()

,날짜,제목,설명,블로거
0,20200111,<b>부산여행</b> 필수코스 / 문화양곱창 / 부산맛집 추천,<b>부산여행</b> 가시는 분들은 문화양곱창 꼭 들리셔서 여행의 특별한 추억 남기...,푸파의 청춘일기
1,20191216,알차게 다녀온 <b>부산여행</b>,오래간만에 다녀온 <b>부산여행</b>은 부산 숨은 곳을 다녀보는 컨셉! 예전에는 ...,미쓰 룰루랄라:)
2,20200108,:: <b>부산여행</b> :: 부산 동백섬 겨울 여행으로 추천드립니다,"<b>부산여행</b> * 부산겨울여행 * 부산동백섬 * 해운대동백섬 여러분, 몇일 ...",류블리공간♡
3,20191210,똑소리 났던 <b>부산여행</b> 맛집,부산에서는 꽤나 인기있는 곳인지라 <b>부산여행</b> 맛집으로도 유명하다고 들었는...,쀠쀠월드+_+
4,20200217,<b>부산여행</b> 범어사 / 홍매화 개화,글/사진 @about_lsj 지금쯤 #<b>부산여행</b> 오신분들은 바다와 더불어...,흰디블로그


In [30]:
# JSON의 딕셔너리의 리스트는 바로 DataFrame으로 처리
df2 = pd.DataFrame(result['items'])
df2.head()

,title,link,description,bloggername,bloggerlink,postdate
0,<b>부산여행</b> 필수코스 / 문화양곱창 / 부산맛집 추천,https://blog.naver.com/kpwee?Redirect=Log&logN...,<b>부산여행</b> 가시는 분들은 문화양곱창 꼭 들리셔서 여행의 특별한 추억 남기...,푸파의 청춘일기,https://blog.naver.com/kpwee,20200111
1,알차게 다녀온 <b>부산여행</b>,https://blog.naver.com/armful31?Redirect=Log&l...,오래간만에 다녀온 <b>부산여행</b>은 부산 숨은 곳을 다녀보는 컨셉! 예전에는 ...,미쓰 룰루랄라:),https://blog.naver.com/armful31,20191216
2,:: <b>부산여행</b> :: 부산 동백섬 겨울 여행으로 추천드립니다,https://blog.naver.com/marzanara?Redirect=Log&...,"<b>부산여행</b> * 부산겨울여행 * 부산동백섬 * 해운대동백섬 여러분, 몇일 ...",류블리공간♡,https://blog.naver.com/marzanara,20200108
3,똑소리 났던 <b>부산여행</b> 맛집,https://blog.naver.com/qo9858?Redirect=Log&log...,부산에서는 꽤나 인기있는 곳인지라 <b>부산여행</b> 맛집으로도 유명하다고 들었는...,쀠쀠월드+_+,https://blog.naver.com/qo9858,20191210
4,<b>부산여행</b> 범어사 / 홍매화 개화,https://blog.naver.com/about_lsj?Redirect=Log&...,글/사진 @about_lsj 지금쯤 #<b>부산여행</b> 오신분들은 바다와 더불어...,흰디블로그,https://blog.naver.com/about_lsj,20200217


# # SOAP

In [ ]:
# 설치는 conda prompt에서 할 것.
!conda install zeep

In [1]:
from zeep import Client, Settings

wsdl = 'http://www.dneonline.com/calculator.asmx?WSDL'
# settings = Settings(strict=False, xml_huge_tree=True)
# client = Client(wsdl, settings=settings)
client = Client(wsdl)
client.service.Add('1', '2')

3